In [1]:
# Libaries
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings("ignore")

print('Libaires imported')

Libaires imported


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s5e6/sample_submission.csv')

print('Data imported')

Data imported


In [3]:
print("Train shape:", train.shape)
print("Test shape:", test.shape)

# Preview the data
display(train.head())
display(test.head())
display(sample_submission.head())

Train shape: (750000, 10)
Test shape: (250000, 9)


,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,750000,31,70,52,Sandy,Wheat,34,11,24
1,750001,27,62,45,Red,Sugarcane,30,14,15
2,750002,28,72,28,Clayey,Ground Nuts,14,15,4
3,750003,37,53,57,Black,Ground Nuts,18,17,36
4,750004,31,55,32,Red,Pulses,13,19,14


,id,Fertilizer Name
0,750000,14-35-14 10-26-26 Urea
1,750001,14-35-14 10-26-26 Urea
2,750002,14-35-14 10-26-26 Urea
3,750003,14-35-14 10-26-26 Urea
4,750004,14-35-14 10-26-26 Urea


In [4]:
# Check column types and missing values
print("\nTrain info:")
print(train.info())
print("\nMissing values in train:")
print(train.isnull().sum())

# Target column value counts
print("\nMost common fertilizers:")
print(train['Fertilizer Name'].value_counts().head(10))

print("\nMost common crop:")
print(train['Crop Type'].value_counts().head(10))


Train info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               750000 non-null  int64 
 1   Temparature      750000 non-null  int64 
 2   Humidity         750000 non-null  int64 
 3   Moisture         750000 non-null  int64 
 4   Soil Type        750000 non-null  object
 5   Crop Type        750000 non-null  object
 6   Nitrogen         750000 non-null  int64 
 7   Potassium        750000 non-null  int64 
 8   Phosphorous      750000 non-null  int64 
 9   Fertilizer Name  750000 non-null  object
dtypes: int64(7), object(3)
memory usage: 57.2+ MB
None

Missing values in train:
id                 0
Temparature        0
Humidity           0
Moisture           0
Soil Type          0
Crop Type          0
Nitrogen           0
Potassium          0
Phosphorous        0
Fertilizer Name    0
dtype: int64

Most common fertil

In [5]:
# Encode target (Fertilizer Name)
le_fert = LabelEncoder()
train['Fert_Label'] = le_fert.fit_transform(train['Fertilizer Name'])

# Encode categorical features: Soil Type, Crop Type
le_soil = LabelEncoder()
le_crop = LabelEncoder()

train['Soil_Type_enc'] = le_soil.fit_transform(train['Soil Type'])
train['Crop_Type_enc'] = le_crop.fit_transform(train['Crop Type'])
test['Soil_Type_enc'] = le_soil.transform(test['Soil Type'])
test['Crop_Type_enc'] = le_crop.transform(test['Crop Type'])

# Feature columns
features = [
    'Temparature', 'Humidity', 'Moisture',
    'Nitrogen', 'Phosphorous', 'Potassium',
    'Soil_Type_enc', 'Crop_Type_enc'
]

# Split data for local validation
X_train, X_val, y_train, y_val = train_test_split(
    train[features], train['Fert_Label'], test_size=0.2, random_state=42
)

# Train a LightGBM model
model = LGBMClassifier()
model.fit(X_train, y_train)

# Predict probabilities for validation
val_probs = model.predict_proba(X_val)
top3_val_preds = np.argsort(-val_probs, axis=1)[:, :3]

# Custom MAP@3 function
def mapk(actual, predicted, k=3):
    score = 0.0
    for a, p in zip(actual, predicted):
        try:
            score += 1.0 / (p[:k].tolist().index(a) + 1)
        except ValueError:
            score += 0.0
    return score / len(actual)

# Evaluate MAP@3 on validation set
map3_val = mapk(y_val.values, top3_val_preds, k=3)
print(f"Validation MAP@3: {map3_val:.4f}")

# Predict probabilities for test set
test_probs = model.predict_proba(test[features])
top3_test_preds = np.argsort(-test_probs, axis=1)[:, :3]

# Convert label indices to fertilizer names row-by-row
top3_labels = np.array([
    le_fert.inverse_transform(row) for row in top3_test_preds
])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011989 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 200
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 8
[LightGBM] [Info] Start training from score -1.885565
[LightGBM] [Info] Start training from score -1.877350
[LightGBM] [Info] Start training from score -1.900582
[LightGBM] [Info] Start training from score -1.909654
[LightGBM] [Info] Start training from score -1.910836
[LightGBM] [Info] Start training from score -2.069993
[LightGBM] [Info] Start training from score -2.091474
Validation MAP@3: 0.3207


In [6]:
# Format the submission
submission = pd.DataFrame({
    "id": test["id"],
    "Fertilizer Name": [" ".join(row) for row in top3_labels]
})

# Save to CSV
submission.to_csv("submission.csv", index=False)